# Lab:  Logistic Regression for Gene Expression Data

In this lab, we use logistic regression to predict biological characteristics ("phenotypes") from gene expression data.  In addition to the concepts in [breast cancer demo](./breast_cancer.ipynb), you will learn to:
* Handle missing data
* Perform multi-class logistic classification
* Create a confusion matrix
* Use L1-regularization for improved estimation in the case of sparse weights (Grad students only)

## Background

Genes are the basic unit in the DNA and encode blueprints for proteins.  When proteins are synthesized from a gene, the gene is said to "express".  Micro-arrays are devices that measure the expression levels of large numbers of genes in parallel.  By finding correlations between expression levels and phenotypes, scientists can identify possible genetic markers for biological characteristics.

The data in this lab comes from:

https://archive.ics.uci.edu/ml/datasets/Mice+Protein+Expression

In this data, mice were characterized by three properties:
* Whether they had down's syndrome (trisomy) or not
* Whether they were stimulated to learn or not
* Whether they had a drug memantine or a saline control solution.

With these three choices, there are 8 possible classes for each mouse.  For each mouse, the expression levels were measured across 77 genes.  We will see if the characteristics can be predicted from the gene expression levels.  This classification could reveal which genes are potentially involved in Down's syndrome and if drugs and learning have any noticeable effects.


## Load the Data

We begin by loading the standard modules.

In [51]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import linear_model, preprocessing

Use the `pd.read_excel` command to read the data from 

https://archive.ics.uci.edu/ml/machine-learning-databases/00342/Data_Cortex_Nuclear.xls

into a dataframe `df`.  Use the `index_col` option to specify that column 0 is the index.  Use the `df.head()` to print the first few rows.

In [47]:
# TODO
df = pd.read_excel('https://archive.ics.uci.edu/ml/machine-learning-databases/00342/Data_Cortex_Nuclear.xls',
                  index_col=0)
df.head(6)

,DYRK1A_N,ITSN1_N,BDNF_N,NR1_N,NR2A_N,pAKT_N,pBRAF_N,pCAMKII_N,pCREB_N,pELK_N,...,pCFOS_N,SYP_N,H3AcK18_N,EGR1_N,H3MeK4_N,CaNA_N,Genotype,Treatment,Behavior,class
MouseID,,,,,,,,,,,,,,,,,,,,,
309_1,0.503644,0.747193,0.430175,2.816329,5.990152,0.218830,0.177565,2.373744,0.232224,1.750936,...,0.108336,0.427099,0.114783,0.131790,0.128186,1.675652,Control,Memantine,C/S,c-CS-m
309_2,0.514617,0.689064,0.411770,2.789514,5.685038,0.211636,0.172817,2.292150,0.226972,1.596377,...,0.104315,0.441581,0.111974,0.135103,0.131119,1.743610,Control,Memantine,C/S,c-CS-m
309_3,0.509183,0.730247,0.418309,2.687201,5.622059,0.209011,0.175722,2.283337,0.230247,1.561316,...,0.106219,0.435777,0.111883,0.133362,0.127431,1.926427,Control,Memantine,C/S,c-CS-m
309_4,0.442107,0.617076,0.358626,2.466947,4.979503,0.222886,0.176463,2.152301,0.207004,1.595086,...,0.111262,0.391691,0.130405,0.147444,0.146901,1.700563,Control,Memantine,C/S,c-CS-m
309_5,0.434940,0.617430,0.358802,2.365785,4.718679,0.213106,0.173627,2.134014,0.192158,1.504230,...,0.110694,0.434154,0.118481,0.140314,0.148380,1.839730,Control,Memantine,C/S,c-CS-m
309_6,0.447506,0.628176,0.367388,2.385939,4.807635,0.218578,0.176233,2.141282,0.195188,1.442398,...,0.109446,0.439833,0.116657,0.140766,0.142180,1.816389,Control,Memantine,C/S,c-CS-m


This data has missing values.  The site:

http://pandas.pydata.org/pandas-docs/stable/missing_data.html

has an excellent summary of methods to deal with missing values.  Following the techniques there, create a new data frame `df1` where the missing values in each column are filled with the mean values from the non-missing values.

In [72]:
# TODO
df1 = df.fillna(df.mean())

## Binary Classification for Down's Syndrome

We will first predict the binary class label in `df1['Genotype']` which indicates if the mouse has Down's syndrome or not.  Get the string values in `df1['Genotype'].values` and convert this to a numeric vector `y` with 0 or 1.  You may wish to use the `np.unique` command with the `return_inverse=True` option.

In [81]:
# TODO
u, indices = np.unique(df1['Genotype'].values, return_inverse = True)
y = indices
print(y)

[0 0 0 ..., 1 1 1]


As predictors, get all but the last four columns of the dataframes.  Standardize the data matrix and call the standardized matrix `Xs`.  The predictors are the expression levels of the 77 genes. 

In [78]:
# TODO
collen = len(df1.columns)
att = df1.columns[:collen-4]
Xs = preprocessing.scale(df1[att])

Create a `LogisticRegression` object `logreg` and `fit` the training data.

In [79]:
# TODO
logreg = linear_model.LogisticRegression(C=1e5)
logreg.fit(Xs, y)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

Measure the accuracy of the classifer.  That is, use the `logreg.predict` function to predict labels `yhat` and measure the fraction of time that the predictions match the true labels.  Below, we will properly measure the accuracy on cross-validation data.

In [80]:
# TODO
yhat = logreg.predict(Xs)
acc = np.mean(yhat == y)
print("Accuracy on training data = %f" % acc)

Accuracy on training data = 1.000000


## Interpreting the weight vector

Create a stem plot of the coefficients, `W` in the logistic regression model.  You can get the coefficients from `logreg.coef_`, but you will need to reshape this to a 1D array.  

In [63]:
# TODO
W = logreg.coef_[0]

[  16.76135724  100.78769901   12.43390757  -28.41889476   -3.13661177
    8.31686459    4.48339598   -7.23332112   16.44125793  -29.52483617
  -16.10991204   11.65602682   29.69027298   -9.21647777   -9.69224115
    7.65403073    4.70974599  -14.43352567  -27.44049414   11.3944864
  -67.05273386  -19.22542662  -18.40367014  -45.69826893  -59.98744224
   17.8109068     2.09228142   11.87741955   20.10203784    1.27185231
   25.31128988    4.95261388   22.06829921  -16.01102568    0.68655865
    3.73202037    9.00451197  -23.37414069   -5.20686433   -7.65245573
   -6.53590684   41.01296015    4.79735591   12.91835668   13.10267379
   13.32555267  -10.75292377    4.18546144    2.24042038    8.56269001
    9.7144092     0.26439961  -13.92265794   -5.82087674   15.55343684
   -3.86288711    4.90784329   -8.94666162  -21.18951067    1.38576505
  -11.14203398   12.39700772   -7.57901984   18.22546039    0.8688318
    4.09998185   -4.9507163   -17.72662673    3.93749257    8.30648489
   -5.82

You should see that `W[i]` is very large for a few components `i`.  These are the genes that are likely to be most involved in Down's Syndrome.  Although, we do not discuss it in this class, there are ways to force the logistic regression to return a sparse vector `W`.  

Find the names of the genes for two components `i` where the magnitude of `W[i]` is largest.  

In [82]:
# TODO
i = np.argsort(-W)[0:2]
print(df1.columns[i])

Index(['ITSN1_N', 'TIAM1_N'], dtype='object')


## Cross Validation

The above meaured the accuracy on the training data.  It is more accurate to measure the accuracy on the test data.  Perform 10-fold cross validation and measure the average precision, recall and f1-score.  Note, that in performing the cross-validation, you will want to randomly permute the test and training sets using the `shuffle` option.  In this data set, all the samples from each class are bunched together, so shuffling is essential.  Print the mean precision, recall and f1-score and error rate across all the folds.

In [84]:
# TODO
from sklearn.model_selection import KFold
from sklearn.metrics import precision_recall_fscore_support
nfold = 10
kf = KFold(n_splits=nfold, shuffle = True)
prec = []
rec = []
f1 = []
acc = []
for train, test in kf.split(Xs):
    Xtr = Xs[train,:]
    ytr = y[train]
    Xts = Xs[test,:]
    yts = y[test]
    
    #fit a model
    logreg.fit(Xtr, ytr)
    yhat = logreg.predict(Xts)
    
    #measure performance
    preci, reci, f1i, _ = precision_recall_fscore_support(yts, yhat, average='binary')
    prec.append(preci)
    rec.append(reci)
    f1.append(f1i)
    acci = np.mean(yhat == yts)
    acc.append(acci)
    
#take average values of the metrics
precm = np.mean(prec)
recm = np.mean(rec)
f1m = np.mean(f1)
accm = np.mean(acc)

# Compute the standard errors
prec_se = np.std(prec)/np.sqrt(nfold-1)
rec_se = np.std(rec)/np.sqrt(nfold-1)
f1_se = np.std(f1)/np.sqrt(nfold-1)
acc_se = np.std(acc)/np.sqrt(nfold-1)

print('Precision = {0:.4f}, SE={1:.4f}'.format(precm,prec_se))
print('Recall =    {0:.4f}, SE={1:.4f}'.format(recm, rec_se))
print('f1 =        {0:.4f}, SE={1:.4f}'.format(f1m, f1_se))
print('Accuracy =  {0:.4f}, SE={1:.4f}'.format(accm, acc_se))

Precision = 0.9521, SE=0.0093
Recall =    0.9503, SE=0.0092
f1 =        0.9509, SE=0.0073
Accuracy =  0.9546, SE=0.0062


## Multi-Class Classification

Now use the response variable in `df1['class']`.  This has 8 possible classes.  Use the `np.unique` funtion as before to convert this to a vector `y` with values 0 to 7.

In [91]:
# TODO
u, indices = np.unique(df1['class'].values, return_inverse = True)
y2 = indices

Fit a multi-class logistic model by creating a `LogisticRegression` object, `logreg` and then calling the `logreg.fit` method.

In [96]:
# TODO
logreg2 = linear_model.LogisticRegression(multi_class='multinomial',
                                          solver='newton-cg',C=1e5)
logreg2.fit(Xs, y)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='multinomial', n_jobs=1, penalty='l2',
          random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
          warm_start=False)

Measure the accuracy on the training data.

In [99]:
# TODO
yhat2 = logreg2.predict(Xs)
acc = np.mean(yhat2==y2)
print("Accuracy on training data = %f" % acc)

Accuracy on training data = 1.000000


Now perform 10-fold cross validation, and measure the confusion matrix `C` on the test data in each fold. You can use the `confustion_matrix` method in the `sklearn` package.  Add the confusion matrix counts across all folds and then normalize the rows of the confusion matrix so that they sum to one.  Thus, each element `C[i,j]` will represent the fraction of samples where `yhat==j` given `ytrue==i`.  Print the confusion matrix.  You can use the command

    print(np.array_str(C, precision=4, suppress_small=True))
    
to create a nicely formatted print.  Also print the overall mean and SE of the test error rate across the folds.

In [131]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold

# TODO
nfold2 = 10
kf2 = KFold(n_splits=nfold2, shuffle = True)
prec2 = []
rec2 = []
f12 = []
acc2 = []
for train, test in kf.split(Xs):
    Xtr2 = Xs[train,:]
    ytr2 = y[train]
    Xts2 = Xs[test,:]
    yts2 = y[test]
    
    #fit a model
    logreg2.fit(Xtr2, ytr2)
    yhat2 = logreg2.predict(Xts2)
    
    #confusion matrix
    c = confusion_matrix(yts2, yhat2)
    csum = np.sum(c, axis=1)
    for i in range(len(c)):
        for j in range(len(c[0])):
            c2[i][j] = c[i][j] / csum[i]
    print(np.array_str(c2, precision=4, suppress_small=True))
    
    #measure performance
    preci, reci, f1i, _ = precision_recall_fscore_support(yts, yhat)
    prec2.append(preci)
    rec2.append(reci)
    f12.append(f1i)
    acci = np.mean(yhat2 == yts2)
    acc2.append(acci)
    
#take average values of the metrics
precm2 = np.mean(prec2)
recm2 = np.mean(rec2)
f1m2 = np.mean(f12)
accm2 = np.mean(acc2)

# Compute the standard errors
prec_se2 = np.std(prec2)/np.sqrt(nfold2-1)
rec_se2 = np.std(rec2)/np.sqrt(nfold2-1)
f1_se2 = np.std(f12)/np.sqrt(nfold2-1)
acc_se2 = np.std(acc2)/np.sqrt(nfold2-1)

print('Precision = {0:.4f}, SE={1:.4f}'.format(precm2,prec_se2))
print('Recall =    {0:.4f}, SE={1:.4f}'.format(recm2, rec_se2))
print('f1 =        {0:.4f}, SE={1:.4f}'.format(f1m2, f1_se2))
print('Accuracy =  {0:.4f}, SE={1:.4f}'.format(accm2, acc_se2))

[[ 0.9524  0.      0.      0.      0.0476  0.      0.      0.    ]
 [ 0.      1.      0.      0.      0.      0.      0.      0.    ]
 [ 0.      0.      1.      0.      0.      0.      0.      0.    ]
 [ 0.      0.      0.      1.      0.      0.      0.      0.    ]
 [ 0.      0.      0.      0.      1.      0.      0.      0.    ]
 [ 0.      0.      0.      0.      0.      1.      0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.      1.      0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.      1.    ]]
[[ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.]]
[[ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  

Re-run the logistic regression on the entire training data and get the weight coefficients.  This should be a 8 x 77 matrix.  Create a stem plot of the first row of this matrix to see the coefficients on each of the genes.

In [134]:
# TODO
logreg3 = linear_model.LogisticRegression(multi_class='multinomial',
                                          solver='newton-cg',C=1e5)
logreg3.fit(Xs, y)
coef = logreg3.coef_
print(coef)

[[  3.22783797e-01  -2.47291633e+00   2.18147440e+00  -6.95209532e-02
    1.79228806e+00  -8.27025773e-02   1.89519464e+00  -4.46053207e+00
   -5.55746235e+00   1.95439277e+00   1.42246687e+00   3.15707743e+00
    2.59156434e+00   5.18799869e-01   2.21299023e+00  -1.69816733e+00
   -4.84645449e+00  -2.97358780e+00   1.65111170e+00   2.40314820e+00
    4.45736744e-01  -5.02199346e-01  -1.21017711e+00   1.97860628e+00
    2.60153253e+00   5.04489556e-01   3.22652663e+00  -1.90288638e+00
   -1.22330944e+00  -4.36298143e-01  -3.19781451e+00  -1.01695002e+00
   -3.59628551e+00   2.90658667e+00  -1.94256064e+00  -1.45714548e-01
   -1.60645542e+00   1.89953980e+00   5.46231457e-01   2.12130808e+00
    4.09368590e-01  -2.77146263e+00  -1.00593988e+00  -1.32460883e+00
   -2.09454268e+00   5.29771730e-01  -2.61341084e-01   1.30296035e+00
   -5.81498111e-01  -5.66958090e-01   6.17622083e-02   2.03731451e+00
   -3.53078947e+00   8.50628898e-01  -4.19663109e-01   4.02250441e+00
   -5.04980609e-01  

## L1-Regularization

Graduate students only complete this section.

In most genetic problems, only a limited number of the tested genes are likely influence any particular attribute.  Hence, we would expect that the weight coefficients in the logistic regression model should be sparse.  That is, they should be zero on any gene that plays no role in the particular attribute of interest.  Genetic analysis commonly imposes sparsity by adding an l1-penalty term.  Read the `sklearn` [documentation](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) on the `LogisticRegression` class to see how to set the l1-penalty and the inverse regularization strength, `C`.

Using the model selection strategies from the [prostate cancer analysis demo](../model_sel/prostate.ipynb), use K-fold cross validation to select an appropriate inverse regularization strength.  
* Use 10-fold cross validation 
* You should select around 20 values of `C`.  It is up to you find a good range.
* Make appropriate plots and print out to display your results
* How does the accuracy compare to the accuracy achieved without regularization.

In [145]:
# TODO
nfold3 = 10
kf3 = KFold(n_splits=nfold3, shuffle = True)
cchoice = np.arange(0, 200, 10)
nd = len(cchoice)
prec3 = np.zeros((nd, nfold3))
rec3 = np.zeros((nd, nfold3))
f13 = np.zeros((nd, nfold3))
acc3 = np.zeros((nd, nfold3))
cchoice = np.arange(0, 200, 10)
for isplit, Ind in enumerate(kf.split(Xs)):
    Itr, Its = Ind
    Xtr3 = Xs[Itr]
    ytr3 = y[Itr]
    Xts3 = Xs[Its]
    yts3 = y[Its]
    
    
    for i, ci in enumerate(cchoice):  
        #fit a model
        logreg3 = linear_model.LogisticRegression(penalty='l1', multi_class='multinomial',
                                          solver='saga',C=ci)
        logreg3.fit(Xtr3, ytr3)
        yhat3 = logreg3.predict(Xts3)
    
        #measure performance
        preci, reci, f1i, _ = precision_recall_fscore_support(yts3, yhat3)
        prec2[i, isplit] = preci
        rec2[i, isplit] = reci
        f12[i, isplit] = f1i
        acci = np.mean(yhat2 == yts2)
        acc2[i, isplit] = acci
    
#take average values of the metrics
precm2 = np.mean(prec2, axis=1)
recm2 = np.mean(rec2, axis=1)
f1m2 = np.mean(f12, axis=1)
accm2 = np.mean(acc2, axis=1)

# Compute the standard errors
prec_se3 = np.std(prec3, axis=1)/np.sqrt(nfold3-1)
rec_se3 = np.std(rec3, axis=1)/np.sqrt(nfold3-1)
f1_se3 = np.std(f13, axis=1)/np.sqrt(nfold3-1)
acc_se3 = np.std(acc3, axis=1)/np.sqrt(nfold3-1)

print('Precision = {0:.4f}, SE={1:.4f}'.format(precm3,prec_se3))
print('Recall =    {0:.4f}, SE={1:.4f}'.format(recm3, rec_se3))
print('f1 =        {0:.4f}, SE={1:.4f}'.format(f1m3, f1_se3))
print('Accuracy =  {0:.4f}, SE={1:.4f}'.format(accm3, acc_se3))

ValueError: b'C <= 0'

For the optimal `C`, fit the model on the entire training data with l1 regularization. Find the resulting weight matrix, `W_l1`.  Plot the first row of this weight matrix and compare it to the first row of the weight matrix without the regularization.  You should see that, with l1-regularization, the weight matrix is much more sparse and hence the roles of particular genes are more clearly visible.

In [17]:
# TODO